<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
SQL: Groupby and Joins
              
</p>
</div>

Data Science Cohort Live NYC Nov 2023
<p>Phase 2</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    
   

In [1]:
import pandas as pd
import sqlite3
import pandas as pd
conn = sqlite3.connect("data/flights.db")
cur = conn.cursor() 

We've seen aggregations last lecture:

In [2]:
# Max value for longitude
pd.read_sql('''
    SELECT 
    
         name, MAX(
            CAST(longitude AS REAL) 
        ) AS max_long
    FROM 
        airports
''', conn)

,name,max_long
0,Moala Airport,179.951


In [ ]:
%%bash

sqlite3 data/flights.db
.schema airlines

In [3]:
# Effectively counts all the active airlines 
pd.read_sql('''
    SELECT 
        COUNT(*) AS number_of_active_airlines
    FROM 
        airlines
    WHERE 
        active='Y'
''', conn)

,number_of_active_airlines
0,1161


This is OK, but want to:
- often compute aggregations across different groups 

#### Grouping in SQL

- Group by values of a categorical column
- Apply aggregation/transformation to groups

GROUP BY statement -- typically used with aggregation!



SELECT colwithgroups,<br>
       agg_function(another_col)<br>
FROM table<br>
GROUP BY colwithgroups<br>

In [4]:
df_results = pd.read_sql('''
    SELECT *
    FROM 
        airlines
        LIMIT 5
''', conn)
df_results

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N


In [5]:
df_results = pd.read_sql('''
    SELECT 
        active, COUNT(*) AS number_of_airlines
    FROM 
        airlines
    GROUP BY
        active
''', conn)

df_results

,active,number_of_airlines
0,N,4886
1,Y,1161
2,n,1


Let's see another example finding the airport with the highest altitude in each country:

In [12]:
df_results = pd.read_sql('''
    SELECT 
        country,
        name as airport_name, MAX(CAST(altitude AS int)) AS altitude
    FROM 
        airports
    GROUP BY 
        country

''', conn)

df_results

,country,altitude
0,Papua New Guinea,933.449365


Let's look at the head of the airlines table:

In [13]:
df_results = pd.read_sql('''
    SELECT *
    FROM 
        airlines 
    LIMIT 3
''', conn)

df_results

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y


**Exercise**

Count the number of active flights grouped by country.

In [14]:
# let's do it!
df_results = pd.read_sql('''
SELECT COUNTRY, COUNT(*) AS COUNTRY_FLIGHTS
FROM airlines
WHERE ACTIVE = "Y"
GROUP BY COUNTRY


    
''', conn)

df_results

,country,COUNTRY_FLIGHTS
0,None,2
1,S.A.,1
2,ALASKA,1
3,AVIANCA,1
4,Afghanistan,4
...,...,...
190,Vietnam,7
191,Yemen,2
192,Zambia,3
193,Zimbabwe,3


**Exercise**

Get the average airport altitude for each country located at latitude > 30 degrees.

In [15]:
# Yaaaaaahhhh!
df_results = pd.read_sql('''
SELECT country, AVG(CAST(altitude as int)) as avg_alt
FROM airports
WHERE CAST(latitude as float) > 30
GROUP BY country

    
''', conn)

df_results

,country,avg_alt
0,Afghanistan,3750.809524
1,Albania,126.000000
2,Algeria,1197.588235
3,Armenia,3905.500000
4,Austria,1332.206897
...,...,...
81,Ukraine,487.939394
82,United Kingdom,141.596154
83,United States,1026.060724
84,Uzbekistan,1149.666667


**Showcase**: let's get fancy pantsy 

- Combining CASE-WHEN and groupbys

- Partition the airports into three latitude regions: 

- northern (>= 30), central (30,-30), and southern zones (<=-30).

Get me the name, altitude, latitude, and country of the airport in each zone with the maximum altitude.

In [16]:
# let's do it!
df_results = pd.read_sql('''

SELECT name AS "Airport Name", country,
       MAX(CAST(altitude AS int)) as max_alt,
       CAST(latitude AS float) AS "Latitude (deg)",
       CASE 
            WHEN CAST(latitude AS float) >= 30 THEN "Northern"
            WHEN CAST(latitude AS float) <= -30 THEN "Southern"
            ELSE "Central"
       END lat_category
       
FROM airports

GROUP BY lat_category

    
''', conn)

df_results

,Airport Name,country,max_alt,Latitude (deg),lat_category
0,Yading Daocheng,China,14472,29.323100,Central
1,Qamdo Bangda Airport,China,14219,30.553600,Northern
2,South Pole Station,Antarctica,9300,-89.999997,Southern


#### Filtering Groups with HAVING

- Can filter columns/tables with WHERE (saw this with groupby above)
- WHERE **doesn't** work when filtering **aggregations** on groupby
- `HAVING` will do the trick. 

Let's only return countries with active airlines having a count greater than 30:

In [17]:
pd.read_sql('''
    SELECT 
        country,
        COUNT(*) AS num
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    HAVING
        num > 30
    ORDER BY 
        num DESC
''', conn)

,country,num
0,United States,141
1,Russia,72
2,United Kingdom,40
3,Germany,37
4,Canada,34


- Can use multiple aggregation functions in groupby.
- Filter on one of the aggregations.

Get the altitude of the highest airport in a country given that it has at least 100 airports.

In [18]:
pd.read_sql('''
    SELECT 
        country,
        COUNT(*) AS num_airports,
        MAX(CAST(altitude as int)) as max_altitude
       
    FROM 
        airports
    GROUP BY 
        country
    HAVING
        num_airports >= 100
''', conn)

,country,num_airports,max_altitude
0,Argentina,103,11414
1,Australia,263,4260
2,Brazil,213,8708
3,Canada,435,3939
4,China,219,14472
5,France,233,6588
6,Germany,321,2297
7,India,140,10682
8,Indonesia,114,9104
9,Japan,131,2200


**Exercise**

List the cities having more than one airport as well as the number of airports the city contains.

Use GROUP BY and HAVING clauses for this.

In [19]:
pd.read_sql('''
SELECT city, COUNT(*) as num_airports

FROM airports

GROUP BY city

HAVING num_airports > 1

''', conn)

,city,num_airports
0,Aachen,2
1,Aberdeen,5
2,Abilene,2
3,Abu Dhabi,3
4,Adana,3
...,...,...
661,Yuryevets,2
662,Zagreb,2
663,Zhuhai,2
664,Zurich,2


#### Joins
- Saw this in Pandas
- Join information from two tables in a relational database together.
- For the two tables: can relate via a common key (aka foreign key).

SQL has many different kinds of joins. We only look at two:
- INNER JOIN
- LEFT JOIN 

A little picture:

<div>
<center><img src="Images/venn.png" align = "center" width="600"/></center>
</div>

#### Inner joins

- Join two tables together on a shared key. 
- Keep rows if the key is in both tables.


SELECT
    table1.column_name, <br>
    table2.different_column_name <br>
FROM <br>
    table1 <br>
    INNER JOIN table2 <br>
        ON table1.shared_column_name = table2.shared_column_name <br>

##### Syntax: Inner Join
SELECT
    table1.column_name, <br>
    table2.different_column_name <br>
FROM <br>
    table1 <br>
    INNER JOIN table2 <br>
        ON table1.shared_column_name = table2.shared_column_name <br>

Let's check out the routes and airline table. 

In [20]:
pd.read_sql('''
    SELECT 
        *
    FROM
        routes
    LIMIT 5
    

''', conn)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2


In [21]:
pd.read_sql('''
    SELECT 
        *
    FROM
        airlines
        LIMIT 5

''', conn)

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N


In [22]:
pd.read_sql('''
    SELECT 
        al.name, al.callsign, rt.airline_id, rt.source, rt.dest
    FROM
        airlines AS al
        INNER JOIN
        routes AS rt
        ON al.id = rt.airline_id
        

''', conn)

,name,callsign,airline_id,source,dest
0,40-Mile Air,MILE-AIR,10,CKX,TKJ
1,40-Mile Air,MILE-AIR,10,FAI,HKB
2,40-Mile Air,MILE-AIR,10,HKB,FAI
3,40-Mile Air,MILE-AIR,10,TKJ,CKX
4,Aigle Azur,AIGLE AZUR,21,AAE,MRS
...,...,...,...,...,...
66980,Rainbow Air Polynesia,Rainbow Air,19676,DAC,ZYL
66981,Rainbow Air Polynesia,Rainbow Air,19676,JSR,DAC
66982,Rainbow Air Polynesia,Rainbow Air,19676,KUL,DAC
66983,Rainbow Air Polynesia,Rainbow Air,19676,SIN,DAC


The airline_id in the routes table and id column in airlines are common keys.

Can join these two tables keeping rows with keys in both tables.

- Note table aliasing

#### LEFT JOIN 

Can join two tables keeping rows with keys in left table only.

In [23]:
pd.read_sql('''
    SELECT 
        *
    FROM
        airlines as al
        
    LEFT JOIN routes as rt
            ON al.id = rt.airline_id
''', conn)

,index,id,name,alias,iata,icao,callsign,country,active,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,1,Private flight,\N,-,None,None,None,Y,NaN,None,None,None,None,None,None,None,None,None
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N,NaN,None,None,None,None,None,None,None,None,None
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y,NaN,None,None,None,None,None,None,None,None,None
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N,NaN,None,None,None,None,None,None,None,None,None
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N,NaN,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72488,6043,19828,Vuela Cuba,Vuela Cuba,6C,6CC,None,Cuba,Y,NaN,None,None,None,None,None,None,None,None,None
72489,6044,19830,All Australia,All Australia,88,8K8,None,Australia,Y,NaN,None,None,None,None,None,None,None,None,None
72490,6045,19831,Fly Europa,None,ER,RWW,None,Spain,Y,NaN,None,None,None,None,None,None,None,None,None
72491,6046,19834,FlyPortugal,None,PO,FPT,FlyPortugal,Portugal,Y,NaN,None,None,None,None,None,None,None,None,None


Can do it the other way:

In [24]:
pd.read_sql('''
    SELECT 
        *
    FROM
        routes as rt
        
    LEFT JOIN airlines as al
            ON rt.airline_id = al.id 
''', conn)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,index,id,name,alias,iata,icao,callsign,country,active
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2,409.0,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2,409.0,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2,409.0,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2,409.0,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2,409.0,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67658,67658,ZL,4178,WYA,6334,ADL,3341,None,0,SF3,4171.0,4178,Regional Express,\N,ZL,RXA,REX,Australia,Y
67659,67659,ZM,19016,DME,4029,FRU,2912,None,0,734,5977.0,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
67660,67660,ZM,19016,FRU,2912,DME,4029,None,0,734,5977.0,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
67661,67661,ZM,19016,FRU,2912,OSS,2913,None,0,734,5977.0,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y


**Exercise**

- Get equipment info (equipment) and destination codes (dest) for American airline ('AA') routes starting from JFK along with latitude, longitude information for JFK.

In [25]:
pd.read_sql('''
SELECT rt.equipment, rt.dest, ap.latitude, ap.longitude
FROM
routes as rt
INNER JOIN 
airports as ap
ON rt.source = ap.code

WHERE ap.code = 'JFK' AND rt.airline = 'AA'

''', conn)

,equipment,dest,latitude,longitude
0,340 330,AMM,40.639751,-73.778925
1,738,ANU,40.639751,-73.778925
2,777,AUH,40.639751,-73.778925
3,738,AUS,40.639751,-73.778925
4,763,BCN,40.639751,-73.778925
...,...,...,...,...
56,330,TXL,40.639751,-73.778925
57,ERD,YUL,40.639751,-73.778925
58,737,YYC,40.639751,-73.778925
59,CR7,YYZ,40.639751,-73.778925


A lot more unique entries on shared key in the airline table vs. the route table.

We won't go through RIGHT JOIN and FULL JOIN here -- its actually fairly self explanatory what they do.

#### Set Operations

- UNION: gets union, drops duplicates
- UNION ALL: gets union, keeps duplicates
- INTERSECT: gets intersection
- EXCEPT: gets what's in one table but not other.

Useful when you have two tables with same data scheme.

- Table1 has some fields with same kind of data as some fields in Table2.
- Can take intersection of fields from two tables with same kind of data, etc.
- Less flexible than joins, but fast and useful in many cases.

<div>
<center><img src="Images/setopd.png" align = "center" width="600"/></center>
</div>

An example might be useful here:

Get countries in airports table not in airlines table.

Countries that have airports but no airlines.

In [26]:
pd.read_sql('''
    SELECT ap.country 
    FROM airports as ap
    EXCEPT
    SELECT al.country
    FROM airlines as al
    LIMIT 10
''', conn)

,country
0,Anguilla
1,Antarctica
2,British Indian Ocean Territory
3,Christmas Island
4,Cocos (Keeling) Islands
5,Dominica
6,East Timor
7,Falkland Islands
8,Gibraltar
9,Greenland


Next up: subqueries!!

Can give us additional flexibility in accessing data!